In [1]:
import json
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import sqlite3
import os

# os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"
tokenizer = AutoTokenizer.from_pretrained(
    "../deepseek-coder-33b-instruct", trust_remote_code=True
)
model = AutoModelForCausalLM.from_pretrained(
    "../deepseek-coder-33b-instruct",
    trust_remote_code=True,
    torch_dtype=torch.bfloat16,
).cuda()

Spider_Dataset_Path = "../dataset/spider/"
Dev_Json_Path = Spider_Dataset_Path + "dev.json"
Database_Path = Spider_Dataset_Path + "database/"

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

: 

: 

In [3]:
# 连接到 SQLite 数据库
def get_schema_str(db_id: str):
    db_path = "../dataset/spider/database/{db_id}/{db_id}.sqlite".format(db_id=db_id)
    conn = sqlite3.connect(db_path)

    # 创建游标对象
    cursor = conn.cursor()

    # 执行查询获取表的模式信息
    cursor.execute("SELECT sql FROM sqlite_master WHERE type='table'")

    # 获取查询结果
    results = cursor.fetchall()

    # # 打印表的模式信息
    # # schema = "```sql\n"
    schema=""
    for result in results:
        schema += result[0].lower() + ";\n"
    # schema+="```\n"
    # 关闭连接
    conn.close()

    return schema

In [33]:
import re
from process_sql import tokenize, get_schema, get_tables_with_alias, Schema, get_sql

pattern = re.compile(r"```sql (.+) ```", re.S)

# \nComplete .
system_prompt = "### Instruction:\nGiven the following SQLite database schema:\n"

# shots=""
# shots = "### Output format rule\nYour answer should only contains sql query in one line in plain text(no markdown format), without any other natrual language. Please follow this rule which is important for me.\n\n"


def is_valid(db_id: str, answer: str):
    db_path = "../dataset/spider/database/{db_id}/{db_id}.sqlite".format(db_id=db_id)
    schema = Schema(get_schema(db_path))
    try:
        p_sql = get_sql(schema, answer)
        return True
    except Exception as e:
        print("invalid")
        print(e)
        return False


def text2sql(
    schema: str,
    question: str,
):
    # conversation = [
    #     {"role": "system", "content": system_prompt+schema},
    #     # {"role": "user", "content": "### Complete sqlite SQL query only and with no explanation \n ### Sqlite SQL tables, with their properties: \n# \n# stadium(Stadium_ID,Location,Name,Capacity,Highest,Lowest,Average)\n;# singer(Singer_ID,Name,Country,Song_Name,Song_release_year,Age,Is_male)\n;# concert(concert_ID,concert_Name,Theme,Stadium_ID,Year)\n;# singer_in_concert(concert_ID,Singer_ID)\n.# \n ### How many singers do we have? \n SELECT"},
    #     # {"role": "system", "content": schema},
    #     {"role": "user", "content": question+"\nSELECT"},
    # ]
    conversation = (
        system_prompt
        + schema
        + "Complete the SQLite SQL query only to answer this question and with no explanation: "
        + question
        + "\n### Response:\nSELECT"
    )
    # print(conversation)
    # print(conv)
    input_text = tokenizer(conversation, return_tensors="pt").to(model.device)
    # input_text = tokenizer.apply_chat_template(conversation, return_tensors="pt").to(model.device)  # type: ignore

    outputs = model.generate(
        **input_text,
        # temperature=1.0,
        # do_sample=True,
        max_new_tokens=512,
        # repetition_penalty=1.5,
        pad_token_id=0,
        bos_token_id=32013,
        eos_token_id=32021,
    )
    # [len(input_text[0]) :]
    answer = tokenizer.decode(
        outputs[0], skip_special_tokens=True
    )
    return answer


fo = open("result.txt", "w")


with open(Dev_Json_Path, "r") as dataset_file:
    i = 0
    dataset = json.load(dataset_file)
    for test_case in dataset[:]:
        i = i + 1
        db_id = test_case["db_id"]
        question = test_case["question"]

        answer = text2sql(get_schema_str(db_id), question)
        # print(answer)

        answer = "SELECT" + answer.replace("\n", " ")
        answer = " ".join(answer.split())
        answer = answer[: answer.find(";") + 1]
        print(answer)
        break
        # continue
        # sql = pattern.findall(answer)
        # if len(sql) > 0:
        #     formatted_answer = sql[0]

        # else:
        #     formatted_answer = answer
        # print(formatted_answer)
        # print(is_valid(db_id, formatted_answer))
        fo.write(answer + "\n")
        # break
        # if i > 10:
        #     break
# 关闭打开的文件
fo.close()

SELECT COUNT(*) FROM singer;
SELECT COUNT(*) FROM singer;
SELECT name, country, age FROM singer ORDER BY age DESC;
SELECT name, country, age FROM singer ORDER BY age DESC;
SELECT AVG(age), MIN(age), MAX(age) FROM singer WHERE country = 'France';
SELECT AVG(age), MIN(age), MAX(age) FROM singer WHERE country = 'France';
SELECT s.name, s.song_release_year FROM singer s WHERE s.age = ( SELECT MIN(age) FROM singer );
SELECT s.name, s.song_name, s.song_release_year FROM singer s WHERE s.age = ( SELECT MIN(s2.age) FROM singer s2 );
SELECT DISTINCT country FROM singer WHERE age > 20;
SELECT DISTINCT s.country FROM singer s WHERE s.age > 20;
SELECT s.country, COUNT(*) as num_singers FROM singer s JOIN singer_in_concert sic ON s.singer_id = sic.singer_id GROUP BY s.country;
SELECT s.country, COUNT(s.singer_id) as singer_count FROM singer s JOIN singer_in_concert sic ON s.singer_id = sic.singer_id GROUP BY s.country;
SELECT s.name, s.song_name FROM singer s JOIN ( SELECT singer_id FROM singer WHE